In [2]:
import pandas as pd
import numpy as np

import random

In [5]:
#Скачать данные с сайта

from src.app import ReestrRequest
from src.queries import lfilt
data = ReestrRequest(lfilt['oil']).get_data_from_reestr()
df = pd.DataFrame(data)

In [7]:
#Загрузить из файла
from analysis.import_path import imp_path
df = pd.read_excel(imp_path())

Извлечение данных с названиями участков (перенесено в app)

In [ ]:
for i in range(100):
    names_df = df.reset_index().set_index('prev_lic')
    names_df = names_df[names_df.index.notnull()]

    #Лист лицензий из names_df: в нем есть названия
    lst = names_df.index.tolist()

    #Лист лицензий из основного ДФ у которых нет названий но есть названия в names_df
    lst2 = df.index[df.index.isin(lst) & df['Наименование участка недр'].isna()].tolist()

    cols = ['Наименование участка недр']

    df.loc[df.index.isin(lst2),cols] = names_df.loc[names_df.index.isin(lst2)]

In [ ]:
len(df['Наименование участка недр']) -  df['Наименование участка недр'].count()

In [ ]:
df[(df['Наименование участка недр'].isna()) & (df.prev_lic.notna())]

In [ ]:
df[df.index.isin(['ЯКУ15826НР','ЯКУ15886НР'])]
#df[df['Наименование участка недр'].isna()]

Подготовка данных для обработки

In [ ]:
#Checks:
max_data = df.loc[df.index.isin(lst2),'Дата лицензии'].max()
ls = df['Наименование участка недр'].unique().tolist()

print(f'Макс возраст лицензии без названия {max_data}')
print(f'Количетсво уникальных названий: {len(ls)}')

print(len(df['Наименование участка недр']) -  df['Наименование участка недр'].count())

PIVOT_CREATING

In [ ]:
df_rind = df.reset_index()
df3 = df_rind.pivot(index=['Наименование участка недр','Государственный регистрационный номер'], columns=['Year'])['prev_lic'].bfill(axis=1)
#df3.loc[['Наименование участка недр']]
df3[df3.index.get_level_values(0).isna()]

-----

In [ ]:
rnd = random.choice(df.index[df['Наименование участка недр'].isna()].tolist())
print (str(rnd))

df3[(~df3.index.get_level_values(0).isna()) & df3.index.get_level_values(1).isin([rnd])]

In [ ]:
df[df.index.isin(['ВЛГ00292НЭ','ВЛГ01572НЭ','ВЛГ00572НЭ','ВЛГ01723НЭ'])]

In [ ]:
df[df.index.isin(['ВЛГ00292НЭ','ВЛГ01572НЭ','ВЛГ00572НЭ','ВЛГ01723НЭ'])]

In [ ]:
df[df.index.isin(['АБН00011НР','АБН00220НП'])]

In [ ]:
#Check indexes
print('----Index level as typle\n', df3.index.levels[1])
print('---- Index level as function:', df3.index.get_level_values(1))

    Обработка дат лицензии и продавцов (Извлечение данных об истории продавцов)

    Сохранение датафреймов как несколько листо в один файл

In [ ]:
path_to_file = 'C:/Users/user/Dropbox/Проекты/Роснедра/Реестр/test.xlsx'
with pd.ExcelWriter(path_to_file, mode='a', if_sheet_exists='replace') as writer:
   #writer.book = openpyxl.load_workbook(path_to_file)
   df2.to_excel(writer, sheet_name='Company_pivot')
   df3.to_excel(writer, sheet_name='Lecension_pivot')